In [2]:
import random
from deap import tools
from deap import base, creator
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

IND_SIZE = 10

toolbox = base.Toolbox()
toolbox.register("attribute", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attribute, n=IND_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evaluate(individual):
    return sum(individual),

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

def main():
    pop = toolbox.population(n=50)
    CXPB, MUTPB, NGEN = 0.5, 0.2, 40

    # Evaluate the entire population
    fitnesses = map(toolbox.evaluate, pop)
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    for g in range(NGEN):
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = map(toolbox.clone, offspring)

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # The population is entirely replaced by the offspring
        pop[:] = offspring

    return pop

main()

TypeError: 'map' object is not subscriptable

In [1]:
import random

from deap import base
from deap import creator
from deap import tools

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

IND_SIZE=10

toolbox = base.Toolbox()
toolbox.register("attr_float", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_float, n=IND_SIZE)
t=toolbox.individual()

In [2]:
print(t)

[0.397106372277641, 0.7770565444639566, 0.3407286569696123, 0.4167457405141243, 0.22168736259763477, 0.4300403971678751, 0.26862635211197083, 0.89264567369053, 0.22777940469207902, 0.0548237043314993]


In [3]:
import operator
print(operator.add(2,3))

5


In [4]:
import random

from deap import base
from deap import creator
from deap import tools

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

IND_SIZE=10

toolbox = base.Toolbox()
toolbox.register("indices", random.sample, range(IND_SIZE), IND_SIZE)
toolbox.register("individual", tools.initIterate, creator.Individual,
                 toolbox.indices)
t=toolbox.individual()
print(t)

[8, 6, 4, 9, 2, 7, 5, 1, 0, 3]


In [16]:
import operator

from deap import base
from deap import creator
from deap import gp
from deap import tools

pset = gp.PrimitiveSet("MAIN", arity=2)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin,
               pset=pset)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual,
                 toolbox.expr)
t=toolbox.individual()
print(t)

sub(sub(ARG0, ARG1), sub(ARG1, ARG0))


In [12]:
print(t(ARG0=5))

TypeError: 'Individual' object is not callable

In [22]:
import array
import random

from deap import base
from deap import creator
from deap import tools

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", array.array, typecode="d",
               fitness=creator.FitnessMin, strategy=None)
creator.create("Strategy", array.array, typecode="d")

def initES(icls, scls, size, imin, imax, smin, smax):
    ind = icls(random.uniform(imin, imax) for _ in range(size))
    ind.strategy = scls(random.uniform(smin, smax) for _ in range(size))
    return ind

IND_SIZE = 10
MIN_VALUE, MAX_VALUE = -5., 5.
MIN_STRAT, MAX_STRAT = -1., 1. 

toolbox = base.Toolbox()
toolbox.register("individual", initES, creator.Individual,
                 creator.Strategy, IND_SIZE, MIN_VALUE, MAX_VALUE, MIN_STRAT, 
                 MAX_STRAT)
t=toolbox.individual()
print(t)
print(t.strategy)
print(t.typecode)

array('d', [-3.712368723412883, 4.520917022304671, 2.851120225254471, -4.314179328534342, -0.482732998263641, 3.9232424186687105, -1.1612074553845209, -3.5520509683388566, -0.10275283141259361, 0.5216253992238737])
array('d', [-0.037658761898096804, -0.7666868172212633, -0.17950771243584263, -0.6138721831518654, 0.6740834742293909, 0.1384916117912678, -0.9448300498292099, -0.13813968662708298, -0.27462661446608894, -0.43800672492914816])
d


In [25]:
import random

from deap import base
from deap import creator
from deap import tools

creator.create("FitnessMax", base.Fitness, weights=(1.0, 1.0))
creator.create("Particle", list, fitness=creator.FitnessMax, speed=None,
               smin=None, smax=None, best=None)

def initParticle(pcls, size, pmin, pmax, smin, smax):
    part = pcls(random.uniform(pmin, pmax) for _ in range(size))
    part.speed = [random.uniform(smin, smax) for _ in range(size)]
    part.smin = smin
    part.smax = smax
    return part

toolbox = base.Toolbox()
toolbox.register("particle", initParticle, creator.Particle, size=2,
                 pmin=-6, pmax=6, smin=-3, smax=3)
t=toolbox.particle()
print(t)
print(t.speed)
print(t.best)
print(t.smin)
print(t.smax)

[-4.913434712250167, -1.0011018881006013]
[-0.08062512784283271, -0.5002738927106045]
None
-3
3


In [51]:
import random

from deap import base
from deap import creator
from deap import tools

IND_SIZE = 5000

creator.create("FitnessMin", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_float", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_float, n=IND_SIZE)

In [52]:
ind1 = toolbox.individual()

In [53]:
print(ind1)              
print(ind1.fitness.valid)

[0.08791657956736942, 0.49580830048673574, 0.5080431548699837, 0.5075264018093807, 0.44001019549453957, 0.06280819099215684, 0.5158297957951197, 0.6482186905091096, 0.7002078659778054, 0.010900570762885309, 0.5898015128471096, 0.5256960174261464, 0.5331902893069856, 0.683196762663734, 0.9183303790702019, 0.8719348731102797, 0.1838849357629464, 0.4022552388924555, 0.34265584289861584, 0.3783252957816625, 0.9782464883836377, 0.23741147475721502, 0.21078797256531911, 0.8682397475802585, 0.4901561996733119, 0.8122042935879938, 0.8691352815234153, 0.6946609078026582, 0.8533483989925492, 0.7269350530926366, 0.9036075594882764, 0.5617235952208302, 0.1863848145672261, 0.4058764800154351, 0.47242539891519164, 0.552293988507254, 0.8126345299633037, 0.17128630379520904, 0.5359932969137149, 0.20890943709688214, 0.10626999868947207, 0.5628187023295579, 0.7807786887484567, 0.25290504138424963, 0.12088217188567385, 0.18043913511727794, 0.6961399300036009, 0.012438077307383777, 0.47852391943062256, 0.

In [54]:
def evaluate(individual):   
    a = sum(individual)
    b = len(individual)
    return a, 1. / b

ind1.fitness.values = evaluate(ind1)
#print(ind1.fitness.valid)
print(ind1.fitness)

(2504.2098056015334, 0.0002)


In [118]:
import random

from deap import base
from deap import creator
from deap import tools

random.seed()

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

sizes=[3,5,3,7]


def initIndividual(indv,sizes):
    vect=[]
    for size_i in sizes:
        vect.append(random.randint(0,size_i-1))
    return indv(vect)

def evaluate(indv):
    return (sum(indv),)

def mutEqua(indv,probmut):
    for i in range(len(indv)):
        if(random.random()<probmut):
            indv[i]=random.randint(0,sizes[i]-1)
        

toolbox = base.Toolbox()
toolbox.register("new_individual",initIndividual,creator.Individual,sizes)
toolbox.register("mate",tools.cxUniform,indpb=0.5)
toolbox.register("mutate",mutEqua,probmut=0.25)
toolbox.register("select",tools.selTournament,tournsize=3)
toolbox.register("new_population",tools.initRepeat,list,toolbox.new_individual)
toolbox.register("evaluate",evaluate)


#t=toolbox.new_individual()
#print(evaluate(t))
#t.fitness.values=evaluate(t)

#mutUniformInt
#cxUniform
#selTournament

In [119]:
NGEN=50
CXPB=0.8
MUTPB=0.1
pop=toolbox.new_population(n=50)
fitnesses=toolbox.map(toolbox.evaluate, pop)
for ind, fit in list(zip(pop, fitnesses)):
    ind.fitness.values = fit
for g in range(NGEN):
    # Select the next generation individuals
    offspring = toolbox.select(pop, len(pop))
    # Clone the selected individuals
    #print(offspring)
    offspring = list(map(toolbox.clone, offspring))

    # Apply crossover on the offspring
    #print(list(offspring))
    #print(offspring[::2])
    #print(offspring[1::2])
    #print(zip(offspring[::2], offspring[1::2]))
    for child1, child2 in list(zip(offspring[::2], offspring[1::2])):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    # Apply mutation on the offspring
    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in list(zip(invalid_ind, fitnesses)):
        ind.fitness.values = fit

    # The population is entirely replaced by the offspring
    pop[:] = offspring

In [117]:
print(pop)

[[2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 0, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [1, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6], [2, 4, 2, 6]]
